<a href="https://colab.research.google.com/github/Yanlingwu95/Pregnancy_Prediction/blob/master/cloud_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 授权绑定Google Drive 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import auth 
auth.authenticate_user() 
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [13]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

# 指定当前的工作目录
import os

# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("drive/CS5412_Cloud_Computing/Digital Agriculture") 


# 查看文件目录，是否包含所需的文件
!ls

'AfiFarm_Daily data_Walnut Ridge_20180819 (April-Aug).xlsx'
'AFI FARM wetransfer-b77a2a  (Aug-Dec).zip'
 cloud_data.ipynb
'Copy of Repro all cows.XLS'
'Data Explanation.pdf'
 drive
'Environmental data.csv'


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [222]:
data = pd.read_excel('./Copy of Repro all cows.XLS')
data_afi = pd.read_excel('./AfiFarm_Daily data_Walnut Ridge_20180819 (April-Aug).xlsx')

data.head(5)


,ID,FDAT,LACT,bolus,DATB1,First Breeding,DIMB1,CODB1,OUTB1,DATB2,...,ABDIM2,ABDAT3,ABDIM3,ABDAT4,ABDIM4,ABDAT5,DCCPR1,DCCPR2,DCCPR3,DCCPR4
0,1,2018-06-04,3,.,2018-08-09,Thu,66,T,O,2018-09-04 00:00:00,...,.,.,.,.,.,.,34DAYS,62DAYS,.,.
1,2,2018-06-04,2,.,2018-08-09,Thu,66,T,O,2018-08-31 00:00:00,...,.,.,.,.,.,.,31DAYS,66DAYS,.,.
2,3,2018-09-17,2,.,2018-11-22,Thu,66,T,O,2018-12-17 00:00:00,...,.,.,.,.,.,.,31DAYS,.,.,.
3,10,2018-12-28,3,1700003117G8V,2019-03-07,Thu,69,T,O,2019-03-14 00:00:00,...,.,.,.,.,.,.,.,.,.,.
4,18,2018-10-09,3,.,2018-12-13,Thu,65,T,P,.,...,.,.,.,.,.,.,32DAYS,60DAYS,.,.


In [223]:
ID = data["ID"].unique()
ID.shape
data["First Breeding"][0]

'Thu'

In [224]:
data_afi.head(5)
id_afi = data_afi["Animal_ID"].unique()
id_afi.shape
col = data.columns

li = []
for s in col:
    li.append(data[s])
len(li[0])

data.iloc[0:3,:].shape
# data_afi["Animal_ID"]


(3, 80)

In [0]:
data_dic = {}
for i in range(ID.shape[0]): 
    data_dic[ID[i]] = [data.iloc[i,:]]

# for i in range(data_afi["Animal_ID"].shape[0]):
#     if data_afi["Animal_ID"][i] in ID:
#         data_dic[data_afi["Animal_ID"][i]].append(data_afi.iloc[i,:])

In [0]:
for i in range(data_afi["Animal_ID"].shape[0]):
    if data_afi["Animal_ID"][i] in ID:
        data_dic[data_afi["Animal_ID"][i]].append(data_afi.iloc[i,:])

In [227]:
def findData(list1, dic, key):
    ind = 8
    listp = []
    listo = []
    while(ind < len(list1)):
        if list1[ind] == 'P':
            listp.append(list1[ind-4])
            break
        elif list1[ind] == 'O':
            listo.append(list1[ind-4])
            ind += 5
        else:
            break
    dic[key] = [listp, listo]

threshold = datetime.datetime(2019, 1, 22, 0, 0) - datetime.datetime(2019, 1, 12, 0, 0)    
time_dic = {}
for key in data_dic:
    findData(data_dic[key][0],time_dic,key)

                    
    

print((time_dic[3]))


        

[[datetime.datetime(2019, 2, 15, 0, 0)], [Timestamp('2018-11-22 00:00:00'), datetime.datetime(2018, 12, 17, 0, 0), datetime.datetime(2019, 1, 22, 0, 0)]]


In [228]:
# print(time_dic[300])
count = 0
def createSet(time_list, afi_table, threshold, dataset):
    
    for li in afi_table:
        
        if time_list[0] != [] and abs(li[2] - time_list[0][0]) < threshold: 
#             print("A")
            dataset.append([li[3:],1])
        else: 
#             print("B")
            for time_o in time_list[1]:
                if abs(li[2] - time_o) < threshold: 
                    dataset.append([li[3:],0])
                    break
                
dataset = []
# createSet(time_dic[1],data_dic[1][1:],threshold, dataset)
for key in time_dic:
    createSet(time_dic[key],data_dic[key][1:],threshold, dataset)
    count += 1
    if (count % 500 == 0):
        print(count)
print(len(dataset))

500
1000
1500
14132


In [229]:
print((dataset[0]))  

[Dry_Days                     0
Days_in_Milk                58
Is_Milked                    1
Lactation_Num                3
Yield (gr)               52840
Conductivity                93
Fat (%)                   3.32
Protein (%)               2.96
Lactose (%)                4.5
Blood (%)                 0.03
SCC (*1000/ml)           56624
Activity (steps/hour)      139
RestTime (minutes)         613
RestBout (#)                12
ProdRate (gr/hour)        2201
RestRatio (%)               43
RestRestlessness            26
RestPerBout (minutes)       51
ActivityDeviation (%)       -4
Name: 14, dtype: object, 0]


In [230]:
import csv
import numpy as np

print(type(dataset))    


data_new = []
for data in dataset:
    temp = []
    for p in data[0]:
        temp.append(p)
    temp.append(data[1])
    data_new.append(temp)
print(len(data_new), len(data_new[0]))
data_set_new = pd.DataFrame(data_new)


<class 'list'>
14132 20


In [231]:
columns_afi = data_afi.columns
columns_afi = columns_afi[3:]
print(type(columns_afi))
columns = ['Dry_Days', 'Days_in_Milk', 'Is_Milked', 'Lactation_Num', 'Yield (gr)',
       'Conductivity', 'Fat (%)', 'Protein (%)', 'Lactose (%)', 'Blood (%)',
       'SCC (*1000/ml)', 'Activity (steps/hour)', 'RestTime (minutes)',
       'RestBout (#)', 'ProdRate (gr/hour)', 'RestRatio (%)',
       'RestRestlessness', 'RestPerBout (minutes)', 'ActivityDeviation (%)','label']
data_set_new.columns = columns

<class 'pandas.core.indexes.base.Index'>


In [232]:
data_set_new.head()

,Dry_Days,Days_in_Milk,Is_Milked,Lactation_Num,Yield (gr),Conductivity,Fat (%),Protein (%),Lactose (%),Blood (%),SCC (*1000/ml),Activity (steps/hour),RestTime (minutes),RestBout (#),ProdRate (gr/hour),RestRatio (%),RestRestlessness,RestPerBout (minutes),ActivityDeviation (%),label
0,0,58,1,3,52840,93.0,3.32,2.96,4.50,0.03,56623.962,139,613,12,2201,43,26,51,-4.0,0
1,0,62,1,3,54376,95.0,3.10,2.90,4.45,0.05,7516.227,138,499,10,2265,35,34,50,4.0,0
2,0,66,1,3,58753,95.0,3.12,2.89,4.56,0.01,13458.609,166,498,13,2448,34,37,38,16.0,0
3,0,70,1,3,57293,96.0,3.23,2.95,4.60,0.04,311.171,141,733,10,2387,53,20,73,1.0,0
4,0,74,1,3,54363,95.0,3.67,2.98,4.44,0.04,30974.205,139,613,10,2265,42,26,61,2.0,0


In [0]:
data_set_new.to_csv("./row_data_sbsb.csv",index=False,sep=',')


In [190]:
dataset_final = dataset[:,0]


TypeError: ignored

In [186]:
dataset_final[1].shape

(21,)

In [114]:
import datetime
print(abs(time_dic[3][1][0] - time_dic[3][1][1]) <threshold)
print(time_dic[3][1])

threshold

False
[Timestamp('2018-11-22 00:00:00'), datetime.datetime(2018, 12, 17, 0, 0), datetime.datetime(2019, 1, 22, 0, 0)]


datetime.timedelta(10)

In [0]:
def dateabs(date1,date2):
  d1 = date1.split("/")
  d2 = date2.split("/")
  if abs(int(d1[0]) - int(d2[0]))<=1 and int(d1[2]) == int(d2[2]):
    return True
  else:
    return False

In [0]:
print (dateabs("04/12/19"))

TypeError: ignored

In [71]:
x = (s for s in range(10))
y = [m for m in range(10)]
print (x)
print (y)
for b in x:
    print (b)


<generator object <genexpr> at 0x7f7f2cddceb8>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
1
2
3
4
5
6
7
8
9
